[Reference](https://medium.com/@datajournal/web-scraping-with-llama-3-520142945316)

In [1]:
!ollama run llama3.1:8b

In [2]:
!ollama serve

/bin/bash: line 1: ollama: command not found


In [4]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 33.9 MB/s eta 0:00:00


In [6]:
!pip install webdriver_manager

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
options = Options()
options.add_argument(" - headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
wait = WebDriverWait(driver, 15)
product_container = wait.until(EC.presence_of_element_located((By.ID, "ppd")))
page_html = product_container.get_attribute("outerHTML")

In [10]:
from markdownify import markdownify as md
clean_text = md(page_html, heading_style="ATX")

In [11]:
PROMPT = (
"You are an expert Amazon product data extractor. Your task is to extract product data from the provided content. "
"Return ONLY valid JSON with EXACTLY the following fields and formats:\n\n"
"{\n"
' "title": "string - the product title",\n'
' "price": number - the current price (numerical value only)",\n'
' "original_price": number or null - the original price if available,\n'
' "discount": number or null - the discount percentage if available,\n'
' "rating": number or null - the average rating (0–5 scale),\n'
' "review_count": number or null - total number of reviews,\n'
' "description": "string - main product description",\n'
' "features": ["string"] - list of bullet point features,\n'
' "availability": "string - stock status",\n'
' "asin": "string - 10-character Amazon ID"\n'
"}\n\n"
"Return ONLY the JSON without any additional text."
)

In [12]:
import requests
import json
response = requests.post(
    "http://localhost:11434/api/generate",
    json={
        "model": "llama3.1:8b",
        "prompt": f"{PROMPT}\n\n{clean_text}",
        "stream": False,
        "format": "json",
        "options": {
        "temperature": 0.1,
        "num_ctx": 12000,
        },
    },
    timeout=250,
)
raw_output = response.json()["response"].strip()
product_data = json.loads(raw_output)

In [13]:
with open("product_data.json", "w", encoding="utf-8") as f:
    json.dump(product_data, f, indent=2, ensure_ascii=False)